<a href="https://colab.research.google.com/github/harjeet88/agentic_ai/blob/main/autogen/gemini_0-6_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -U -q autogen-agentchat autogen autogen-ext[openai] google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 838.3/838.3 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.4 MB/s eta 0:00:00


In [5]:
import autogen
import google.generativeai as genai
from autogen import AssistantAgent, UserProxyAgent
from autogen_ext.models import *

In [6]:
gemini_flash="gemini-2.0-flash"
gemma_1b="gemma-3-1b-it"
gemma_4b="gemma-3-4b-it"
gemma_12b="gemma-3-12b-it"
gemma_27b="gemma-3-27b-it"
gemma_e4b="gemma-3n-e4b-it"
llama3="llama3.2:1b"

In [8]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
GEMINI_API_KEY=userdata.get('GOOGLE_API_KEY')

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import ModelInfo,ModelFamily
from autogen_core.models import UserMessage
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_agentchat.conditions import TextMentionTermination,MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat

In [11]:
gemini_base_url="https://generativelanguage.googleapis.com/v1beta/openai/"

In [10]:
model_info = {
        "family": ModelFamily.GEMINI_2_5_FLASH,
        "function_calling": True,
        "json_output": True,
        "vision": False
    }

In [13]:
gemini_client = OpenAIChatCompletionClient(
    model=gemini_flash,
    base_url=gemini_base_url,
    model_info=model_info,#ModelInfo(vision=True, function_calling=True, json_output=True, family="unknown", structured_output=True),
    api_key=GEMINI_API_KEY
    # api_key="sk-...", # Optional if you have an OPENAI_API_KEY set in the environment.
)

/usr/local/lib/python3.11/dist-packages/autogen_ext/models/openai/_openai_client.py:439: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


In [16]:
agent = AssistantAgent("assistant", gemini_client)
print(await agent.run(task="Say 'Hello World!'"))

messages=[TextMessage(id='f6118379-f238-47fc-86f3-da9e0d5a5f29', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 2, 9, 24, 54, 995843, tzinfo=datetime.timezone.utc), content="Say 'Hello World!'", type='TextMessage'), TextMessage(id='b33f998a-0558-4e89-9c4b-e4099f6d7b9d', source='assistant', models_usage=RequestUsage(prompt_tokens=29, completion_tokens=7), metadata={}, created_at=datetime.datetime(2025, 7, 2, 9, 24, 55, 666547, tzinfo=datetime.timezone.utc), content='Hello World!\nTERMINATE\n', type='TextMessage')] stop_reason=None


In [19]:
agent = AssistantAgent("assistant", gemini_client)
await Console(agent.run_stream(task="What is the capital of New York?"))

---------- TextMessage (user) ----------
What is the capital of New York?
---------- TextMessage (assistant) ----------
The capital of New York is Albany.
TERMINATE



TaskResult(messages=[TextMessage(id='b41ed067-e88e-4754-bc59-7ca0c05fbc66', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 2, 9, 25, 46, 636531, tzinfo=datetime.timezone.utc), content='What is the capital of New York?', type='TextMessage'), TextMessage(id='dd9a7c57-e272-4307-bc3d-477568deeefe', source='assistant', models_usage=RequestUsage(prompt_tokens=32, completion_tokens=12), metadata={}, created_at=datetime.datetime(2025, 7, 2, 9, 25, 47, 234699, tzinfo=datetime.timezone.utc), content='The capital of New York is Albany.\nTERMINATE\n', type='TextMessage')], stop_reason=None)

In [27]:
from autogen_agentchat.agents import UserProxyAgent

user_proxy = UserProxyAgent("user_proxy")

In [22]:
travel_agent = AssistantAgent(
    "assitant_agent",
    model_client=gemini_client,
    #handoffs=["flights_refunder", "user"],
    system_message="""You are a travel agent.
    The flights_refunder is in charge of refunding flights.
    If you need information from the user, you must first send your message, then you can handoff to the user.
    Use TERMINATE when the travel planning is complete.""",
)

In [24]:
# Define a termination condition that stops the task if the critic approves or after 10 messages.
termination = TextMentionTermination("APPROVE") | MaxMessageTermination(10)

In [28]:
team = RoundRobinGroupChat([user_proxy, travel_agent], termination_condition=termination)

In [30]:
await Console(team.run_stream(task="i want to travel to japan. help me"))

---------- TextMessage (user) ----------
i want to travel to japan. help me
---------- TextMessage (assitant_agent) ----------
Okay, great! Just need a little more information:

1.  **What are your interests?** (e.g., culture, food, nature, anime, history, etc.)

Enter your response: nature
---------- TextMessage (user_proxy) ----------
nature
---------- TextMessage (assitant_agent) ----------
Okay, a luxurious two-week trip to Japan focused on nature in August! That sounds amazing.

Given your interest in nature and a luxury budget, I recommend focusing on these regions and experiences:

*   **Japanese Alps (Chubu Region):** Hike in stunning mountain scenery, visit tranquil alpine villages like Kamikochi, and stay in luxurious onsen (hot spring) resorts with mountain views.
*   **Hokkaido:** Escape the summer heat and explore Hokkaido's national parks, flower fields (lavender will be in bloom!), and enjoy fresh seafood. Consider a private tour or a stay in a high-end ryokan (tradition

TaskResult(messages=[TextMessage(id='5c491adc-3161-402c-be8e-f2abbd049a83', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 2, 9, 41, 46, 317880, tzinfo=datetime.timezone.utc), content='i want to travel to japan. help me', type='TextMessage'), TextMessage(id='dfe1a0f5-cad3-4ac3-98de-52e6a96ba9b7', source='assitant_agent', models_usage=RequestUsage(prompt_tokens=423, completion_tokens=40), metadata={}, created_at=datetime.datetime(2025, 7, 2, 9, 41, 47, 317199, tzinfo=datetime.timezone.utc), content='Okay, great! Just need a little more information:\n\n1.  **What are your interests?** (e.g., culture, food, nature, anime, history, etc.)\n', type='TextMessage'), UserInputRequestedEvent(id='a425384c-ff38-47aa-9848-0597a3b96dc4', source='user_proxy', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 2, 9, 41, 47, 319701, tzinfo=datetime.timezone.utc), request_id='2b8c8820-d3ae-4e3f-84d4-9e88d6fbab2d', content='', type='UserInputReques

#rag example